In [10]:
import numpy as np
import joblib
import os
from pathlib import Path
import tensorflow as tf
from Utils.Data_Preprocessing import *
from sklearn.preprocessing import MinMaxScaler

def load_models():
    """
    Load both .pkl and .h5 models from the Models directory.
    Simply loads and prints model types without doing any inference.
    """
    # Define path to your Models directory
    project_root = Path('.')  # Current directory
    models_dir = project_root / 'Models'
    
    # Check if directory exists
    if not models_dir.exists():
        print(f"Error: Models directory not found at {models_dir}")
        return None
    
    print(f"Loading models from: {models_dir}")
    
    # Define model files based on your directory structure
    pkl_models = [
        'decision_tree_model.pkl',
        'linear_regression_model.pkl',
        'prophet_model.pkl',
        'random_forest_model.pkl',
        'slinear_regression_model.pkl',
        'svr_model.pkl',
        'voting_model.pkl',
        'xgboost_model.pkl'
    ]
    
    h5_models = [
        'gru_model.h5',
        'lstm_model.h5'
    ]
    
    # Dictionary to store loaded models
    loaded_models = {}
    
    # Load pickle models
    print("\n--- Loading Pickle Models ---")
    for model_file in pkl_models:
        model_path = models_dir / model_file
        try:
            # Only try to load if file exists
            if model_path.exists():
                model = joblib.load(model_path)
                model_name = model_path.stem  # Get filename without extension
                loaded_models[model_name] = model
                
                # Check model type
                model_type = type(model).__name__
                has_predict = hasattr(model, 'predict')
                
                print(f"✓ {model_name}: Type={model_type}, Has predict method={has_predict}")
            else:
                print(f"⚠ {model_file} not found, skipping")
        except Exception as e:
            print(f"✗ Error loading {model_file}: {e}")
    
    # Load h5 models (TensorFlow/Keras models)
    print("\n--- Loading H5 Models ---")
    for model_file in h5_models:
        model_path = models_dir / model_file
        try:
            # Only try to load if file exists
            if model_path.exists():
                # TensorFlow requires string path
                model = tf.keras.models.load_model(str(model_path))
                model_name = model_path.stem  # Get filename without extension
                loaded_models[model_name] = model
                
                # Check model type and summary
                model_type = type(model).__name__
                has_predict = hasattr(model, 'predict')
                
                print(f"✓ {model_name}: Type={model_type}, Has predict method={has_predict}")
                
                # Print model summary
                print(f"Model structure:")
                model.summary()
            else:
                print(f"⚠ {model_file} not found, skipping")
        except Exception as e:
            print(f"✗ Error loading {model_file}: {e}")
    
    print(f"\nSuccessfully loaded {len(loaded_models)} models")
    return loaded_models

In [11]:
loaded_models= load_models()

Loading models from: Models

--- Loading Pickle Models ---
✓ decision_tree_model: Type=DecisionTreeRegressor, Has predict method=True
✓ linear_regression_model: Type=LinearRegression, Has predict method=True
✓ prophet_model: Type=Prophet, Has predict method=True
✓ random_forest_model: Type=RandomForestRegressor, Has predict method=True
⚠ slinear_regression_model.pkl not found, skipping
✓ svr_model: Type=SVR, Has predict method=True
✓ voting_model: Type=VotingRegressor, Has predict method=True
✓ xgboost_model: Type=XGBRegressor, Has predict method=True

--- Loading H5 Models ---
✓ gru_model: Type=Sequential, Has predict method=True
Model structure:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 5, 150)         │        70,650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 5, 150)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 5, 200)         │       211,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 5, 200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 150)            │       158,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 96)             │        14,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            97 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 454,844 (1.74 MB)

 Trainable params: 454,843 (1.74 MB)

...

 Total params: 387,331 (1.48 MB)

 Trainable params: 387,329 (1.48 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)


Successfully loaded 9 models


## Classical Model Inference

In [12]:
def inference_classical_models(data):
    import numpy as np
    import pandas as pd
    import joblib

    # Load the scalers
    scaler_X = joblib.load('Scaler/scaler_X_minmax.pkl')
    scaler_y = joblib.load('Scaler/scaler_y_minmax.pkl')
    #model = joblib.load('Models/linear_regression_model.pkl')
    
    
    # Filter only classical models (skip keras models which are of type 'Functional', 'Sequential', etc.)
    classical_models = {
        name: model for name, model in loaded_models.items()
        if not isinstance(model, tf.keras.Model)
    }

    # # Read last 20 rows (not 19!) for moving average support
    # df_hist = pd.read_csv('Data/Tasla_Stock_Updated_V2.csv').tail(20)

    # # Add the new data (must match expected column names)
    # df_new = pd.DataFrame([data], columns=['Low', 'High', 'Open', 'Close', 'Volume'])
    # df_combined = pd.concat([df_hist, df_new], ignore_index=True)
    
    # # Select features for prediction (same as used during training)
    # features = ['Low', 'High', 'Open', 'Close', 'Volume']
    # # Extract the last row (new data) with features
    # X = df_combined[features].iloc[-1:].values  # Shape: (1, n_features)

    # Scale the features
    X_scaled = scaler_X.transform(np.array(data).reshape(1, -1))

    predictions = {}

    # Inference using each classical model
    for name, model in classical_models.items():
        try:
            y_pred_scaled = model.predict(X_scaled)
            y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
            predictions[name] = y_pred[0][0]
            print(f"✓ {name} prediction: {y_pred[0][0]:.4f}")
        except Exception as e:
            print(f"✗ Error predicting with {name}: {e}")

    return predictions
    


inference_classical_models([23.543333053588867,23.83133316040039,22.858667373657227,22.99933242797852,114088500])
inference_classical_models( [347.7366638183594,349.4800109863281,340.8133239746094,341.8299865722656,55013700])

✓ decision_tree_model prediction: 23.0167
✓ linear_regression_model prediction: 23.1745
✗ Error predicting with prophet_model: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
✓ random_forest_model prediction: 22.8487
✓ svr_model prediction: 46.4411
✓ voting_model prediction: 28.8450
✓ xgboost_model prediction: 22.9157
✓ decision_tree_model prediction: 331.8833
✓ linear_regression_model prediction: 342.9801
✗ Error predicting with prophet_model: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
✓ random_forest_model prediction: 339.9581
✓ svr_model prediction: 351.0005
✓ voting_model prediction: 341.3777
✓ xgboost_model prediction: 331.5721


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/

{'decision_tree_model': 331.8833312988281,
 'linear_regression_model': 342.98010733496636,
 'random_forest_model': 339.95813232421875,
 'svr_model': 351.00048754605604,
 'voting_model': 341.3777125941356,
 'xgboost_model': 331.5721}

## LSTM models inferencing

In [20]:
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf

def inference_lstm_models_with_history(single_row, window_size=5, tweak_pct=0.01):
    """
    Perform inference for LSTM/GRU models using a sequence made from
    a synthetic sequence of the latest (window_size - 1) rows slightly tweaked from
    the input row + current input row.

    Args:
        single_row (list or np.array): Latest input features (1 timestep).
        dataset_path (str): Path to the CSV dataset file (only used for scaler loading).
        window_size (int): Number of timesteps expected by the model.
        tweak_pct (float): Max percentage tweak (+/-) to apply to generate synthetic previous steps.

    Returns:
        dict: Predictions from each LSTM/GRU model.
    """

    # Load scalers
    scaler_X = joblib.load('Scaler/scaler_X_minmax.pkl')
    scaler_y = joblib.load('Scaler/scaler_y_minmax.pkl')

    # Assuming loaded_models is globally defined or imported:
    rnn_models = {
        name: model for name, model in loaded_models.items()
        if isinstance(model, tf.keras.Model)
    }

    features = ['Open', 'High', 'Low', 'Close', 'Volume']

    # Create synthetic sequence by tweaking single_row for previous (window_size - 1) timesteps
    synthetic_sequence = []
    for i in range(window_size - 1):
        # Apply random tweak within ±tweak_pct
        tweak_factors = 1 + np.random.uniform(-tweak_pct, tweak_pct, size=len(single_row))
        synthetic_step = single_row * tweak_factors
        synthetic_sequence.append(synthetic_step)

    # Convert list to np.array
    synthetic_sequence = np.array(synthetic_sequence)

    # Append the original single_row as the last timestep
    full_sequence = np.vstack([synthetic_sequence, single_row])

    # Scale the sequence
    seq_scaled = scaler_X.transform(full_sequence)

    # Reshape for model input (1, window_size, features)
    seq_scaled = seq_scaled.reshape(1, window_size, len(features))

    predictions = {}
    for name, model in rnn_models.items():
        try:
            y_pred_scaled = model.predict(seq_scaled)
            y_pred = scaler_y.inverse_transform(y_pred_scaled)
            predictions[name] = y_pred[0][0]
            print(f"✓ {name} prediction: {y_pred[0][0]:.4f}")
        except Exception as e:
            print(f"✗ Error predicting with {name}: {e}")

    return predictions


# Example usage
input_row = np.array([15.658666610717772,17.031999588012695,14.642666816711426,14.732666969299316,52843500])
dataset_path = 'Data/Tasla_Stock_Updated_V2.csv'  # Only used for scaler loading here

predictions = inference_lstm_models_with_history(input_row, window_size=5, tweak_pct=0.1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
✓ gru_model prediction: 21.2220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
✓ lstm_model prediction: 15.3399


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
